In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import pandas as pd


In [55]:
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\combine_all_class_with_label_no_fft.csv'
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\combine_all_class_with_label_apply_fft.csv'
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\cleaned_combine_all_class_with_label_applied_fft.csv'
file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win5_lab50.csv'
df = pd.read_csv(file_path)

In [56]:
# -- Prepare for modeling --
# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
#    - We'll drop 'start_time' because it's just a reference
#    - We'll keep 'label_encoded' as the target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
y = df['label_encoded']

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,   # 20% for testing
    random_state=42  # for reproducibility
)

# Apply min-max normalization (0-1) to these columns
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# 4) Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5) Evaluate
y_pred = model.predict(scaler.transform(X_test))
print(classification_report(y_test, y_pred, target_names=le.classes_))

                             precision    recall  f1-score   support

            climbing_stairs       0.84      0.72      0.78        53
          descending_stairs       0.90      0.88      0.89        73
                    nothing       1.00      0.93      0.96        54
                    running       0.98      1.00      0.99        53
sitting_standing_transition       0.93      0.98      0.95        64
                    walking       0.86      0.97      0.91        67

                   accuracy                           0.91       364
                  macro avg       0.92      0.91      0.91       364
               weighted avg       0.92      0.91      0.91       364



In [ ]:
!pip install tensorflow onnx onnxmltools skl2onnx nyoka scikit-learn joblib pandas numpy onnxruntime jpmml-evaluator

^C


In [96]:
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
import onnx
import onnxmltools
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from nyoka import skl_to_pmml
import json

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# -- Load Data --
file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win2_lab50.csv'
df = pd.read_csv(file_path)

# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
y = df['label_encoded']

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_test_raw = X_test
X_train_raw = X_train
# 4) Normalize features (0-1 scaling)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5) Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# 6) Save the trained model
# Extract the parameters from the scaler
scaler_params = {
    "data_min": scaler.data_min_.tolist(),
    "data_max": scaler.data_max_.tolist(),
    "scale": scaler.scale_.tolist(),
    "min": scaler.min_.tolist()
}

# Save the parameters as a JSON file
# with open("scaler_params.json", "w") as f:
#     json.dump(scaler_params, f)


# 7) Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))


                             precision    recall  f1-score   support

            climbing_stairs       0.85      0.78      0.82       170
          descending_stairs       0.83      0.81      0.82       158
                    nothing       1.00      0.91      0.95       148
                    running       1.00      1.00      1.00       145
sitting_standing_transition       0.91      0.99      0.94       146
                    walking       0.85      0.94      0.89       154

                   accuracy                           0.90       921
                  macro avg       0.91      0.91      0.90       921
               weighted avg       0.90      0.90      0.90       921



In [6]:
# IF USE TORCH
# import torch
# import torch.nn as nn
# import torch.optim as optim

# class RF_NN(nn.Module):
#     def __init__(self, input_size, num_classes):
#         super(RF_NN, self).__init__()
#         self.fc1 = nn.Linear(input_size, 160)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(160, num_classes)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         x = self.softmax(x)
#         return x

# # Create the PyTorch model
# input_size = X_train.shape[1]
# num_classes = len(set(y))
# torch_model = RF_NN(input_size, num_classes)


In [147]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# -- Load Data --
file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win2_lab50.csv'
df = pd.read_csv(file_path)

# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
y = df['label_encoded']

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_test_raw = X_test
X_train_raw = X_train
# 4) Normalize features (0-1 scaling)
scaler = MinMaxScaler()
scaler.fit(X_train)

# Convert integer labels to one-hot encoded labels for MLP training
num_classes = len(le.classes_)
y_train_ohe = to_categorical(y_train, num_classes=num_classes)
y_test_ohe = to_categorical(y_test, num_classes=num_classes)

# Create constant tensors for the normalization parameters
scaler_scale = tf.constant(scaler.scale_, dtype=tf.float32)
scaler_min = tf.constant(scaler.min_, dtype=tf.float32)

def rf_to_nn_with_normalization(activation_fn:str):
    return tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train_raw.shape[1],)),
        tf.keras.layers.Lambda(lambda x: x * scaler_scale + scaler_min, name="Normalization"),
        tf.keras.layers.Dense(85, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(65, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(45, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(35, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(25, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(15, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(10, activation=activation_fn,
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(num_classes, activation="softmax",
                              kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED))
    ])

# Create and compile the model with normalization incorporated
tf_model = rf_to_nn_with_normalization('relu')
tf_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
tf_model.summary()

# Train the model using one-hot encoded labels
# Note: Since the model applies internal normalization, we feed raw features.
tf_model.fit(
    X_train_raw, 
    y_train_ohe,  
    epochs=63, 
    verbose=1
)

# Convert the trained model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
tflite_model = converter.convert()
tflite_model_path = "random_forest.tflite"

# Save the TFLite model to file
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved: {tflite_model_path}")


Model: "sequential_55"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Normalization (Lambda)          │ (None, 51)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_313 (Dense)               │ (None, 85)             │         4,420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_314 (Dense)               │ (None, 65)             │         5,590 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_315 (Dense)               │ (None, 45)             │         2,970 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_316 (Dense)               │ (None, 35)             │         1,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_317 (Dense)               │ (None, 25)             │           900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_318 (Dense)               │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_319 (Dense)               │ (None, 10)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_320 (Dense)               │ (None, 6)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,106 (62.91 KB)

 Trainable params: 16,106 (62.91 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3752 - loss: 1.5995
Epoch 2/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6526 - loss: 0.7866
Epoch 3/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7619 - loss: 0.5662
Epoch 4/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8000 - loss: 0.4898
Epoch 5/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8175 - loss: 0.4494
Epoch 6/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8332 - loss: 0.4219
Epoch 7/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8391 - loss: 0.4010
Epoch 8/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8454 - loss: 0.3847
Epoch 9/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8572 - loss: 0.3695
Epoch 10/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8669 - loss: 0.3517
Epoch 11/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8701 - loss: 0.3413
Epoch 12/63
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

INFO:tensorflow:Assets written to: C:\Users\tanar\AppData\Local\Temp\tmppd7fvnpa\assets


Saved artifact at 'C:\Users\tanar\AppData\Local\Temp\tmppd7fvnpa'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 51), dtype=tf.float32, name='keras_tensor_533')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  2236045604848: TensorSpec(shape=(51,), dtype=tf.float32, name=None)
  2236061331728: TensorSpec(shape=(51,), dtype=tf.float32, name=None)
  2236137222064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2236137305920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2236137306624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2236137328832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2236137328304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2236137324784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2236137329888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2236137324608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  223613

In [149]:
# ✅ **Test TFLite Model**
import tensorflow.lite as tflite

tflite_model_path = "random_forest.tflite"
interpreter = tflite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

tflite_preds = []
for sample in X_test_raw.values.astype(np.float32):
    sample = np.expand_dims(sample, axis=0)
    interpreter.set_tensor(input_index, sample)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_index)
    tflite_preds.append(np.argmax(pred))

print("TFLite Model Performance:\n", classification_report(y_test, tflite_preds, target_names=le.classes_))



TFLite Model Performance:
                              precision    recall  f1-score   support

            climbing_stairs       0.85      0.89      0.87       170
          descending_stairs       0.87      0.87      0.87       158
                    nothing       1.00      0.93      0.96       148
                    running       1.00      0.99      0.99       145
sitting_standing_transition       0.91      0.97      0.94       146
                    walking       0.94      0.92      0.93       154

                   accuracy                           0.93       921
                  macro avg       0.93      0.93      0.93       921
               weighted avg       0.93      0.93      0.93       921

